In [ ]:
!pip install transformers[sentencepiece] 

In [ ]:
import torch
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu" 
classifier = pipeline("zero-shot-classification",
                      model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
                      device=device)

In [ ]:
with open('profanity.txt', 'r', encoding='utf-8') as file:
    words = list(line.strip().lower() for line in file)

In [ ]:
list_label = [
    "позитив", 
    "негатив", 
    "политика", 
    "веселье", 
    "религия", 
    "секс", 
    "насилие", 
    "наркотики", 
    "медицина", 
    "алкоголь", 
    "спорт", 
    "суицид", 
    "наука", 
    "путешествия", 
    'дипрессия',
    'творчество',
    'дискриминация',
    'сексизм',
    'травля',
    'юмор',
    'кровь',
    'смех',
    'убийство',
    'история'
]
forbidden_theme = ['наркотики', 'алкоголь', 'суицид', 'насилие', 'секс', 'религия', 'политика', 'дипрессия', 'убийство', 'кровь', 'травля', 'сексизм', 'дискриминация']

# forbidden_labels = [
#     "политика",
#     "религия",
#     "секс",
#     "насилие",
#     "наркотики",
#     "алкоголь",
#     "ненависть",
#     "дискриминация",
#     "расизм",
#     "сексизм",
#     # "предрассудки",
#     "суицид",
#     # "кибербуллинг",
# #     "нецензурная лексика",
#     # "контент для взрослых",
#     # "азартные игры",
#     "незаконная деятельность",
#     "депрессия"
# ]
# permitted_labels = [
#     "позитив",
#     "негатив",
#     "развлечение",
#     "технологии",
#     "образование",
#     "наука",
#     "искусство",
#     "культура",
#     "здоровье",
#     # "спорт",
#     "путешествия",
#     "природа",
#     # "животные",
#     "питание",
#     # "мода",
#     # "музыка",
#     # "литература",
#     "языки"
# ]

In [ ]:
def check_profanity(text, profanity_list):
    """Проверяет есть ли в тексте запрещенные слова

    Args:
        text (str): Текст для проверки
        profanity_list (_type_): список запрещенных слов

    Returns:
        (str|None): Возвращает текст если в нём нет запрещенных слов, иначе воспроизводит случайный ответ из набора данных и возвращает None
    """
    import random 
    import re 
    
    answers = [
    'В твоем вопросе плохое слово',
    'Ты что, это же нехорошее слово.',
    'Мне не послышалось? Ты это вслух сказал?',
    'Не ругайся, а то расскажу твоим родителям',
    'Сквернословие тебя не красит',
    'Ругаться это совсем не круто']
    
    pattern = r'\b({})\b'.format('|'.join(map(re.escape, profanity_list)))
    if re.search(pattern, text, flags=re.IGNORECASE):
        print(random.choice(answers))
        return 
    return text

In [ ]:
def text_classification(text, model, list_label, forbidden_theme):
    """Функция для классификации текста

    Args:
        text (str): Текст для дальнейшей обработки
        model (_type_): модель классификатор
        list_label (list): Список всех тем
        forbidden_theme (list): список запрещенных тем

    Returns:
        (str|None): Возвращает текст если всё хорошо, если присутствует запрещенная тема или мат воспроизводит случайный текст из набора и возвращает None.
    """
    import random
    answers = [
    'Это тайна, которую я еще не разгадал.',
    'Если бы я знал, я бы уже рассказал тебе.',
    'Затрудняюсь ответить.',
    'Ой наверно про такое непристойно спрашивать.',
    'Если бы я был чуточку уменее, то ответил на твой вопрос, а так я очень умный',
    'Понятия не имею.',
    'Мне кажется, это слишком сложный вопрос для тебя. Давай я попробую ответить на более простой.',
    'Давай лучше поговорим о чем-нибудь другом, этот вопрос мне не нравится.',
    'Я в этой теме не разбираюсь, лучше спроси у взрослых.',
    'Боюсь, тут я бессилен',
    'Я могу рассказать только то, что знаю сам.',
    'К сожалению, я не знаю ответа.',
    'Ой, это взрослые знают',
    'Незнаю, я ведь Незнайка',
    'Я не расслышал, спроси ещё раз']
    
    if not check_profanity(text, words):
        return
    res = model(text, list_label, multi_label=False)
    sum_proba_forbidden_theme = sum(score for label, score in zip(res['labels'], res['scores']) if label in forbidden_theme)
    if sum_proba_forbidden_theme > 1 - sum_proba_forbidden_theme:
        print(random.choice(answers))
        return
    return text